In [ ]:
import cmeutils
from cmeutils import writers
import cmeutils.polymers as polymer
from cmeutils.gsd_utils import snap_molecule_cluster
import warnings
warnings.filterwarnings("ignore")
import time
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
import mbuild as mb
import gsd
import gsd.hoomd

## Example

This example is for a .gsd file that contains 10 molecuels, each one with 5 monomers
The molecules are united-atom PEEK, so each monomer contains 22 atoms

In [ ]:
gsdfile = "100-20mers.gsd"
peek_system = polymer.System(gsd_file=gsdfile, gsd_frame=0, atoms_per_monomer=22)

# Hierarchial structures




In [ ]:
print(f"{peek_system.n_molecules} Molecules")
print(f"{peek_system.n_monomers} Monomers")
print(f"{peek_system.n_atoms} Atoms")
print()
print(len((peek_system.molecules)))
print(peek_system.molecules[0:5])

### Looking at just 1 of the molecule objects from peek_system.molecules

In [ ]:
molecule_1 = peek_system.molecules[0]
print(f"{molecule_1.n_monomers} Monomers")
print(f"{molecule_1.n_atoms} Atoms")
print()
print(len(molecule_1.monomers))
print(molecule_1.monomers[0:5])

### Looking at just 1 of the monomer objects from molecule_1

In [ ]:
monomer_1 = molecule_1.monomers[0]
print(f"{monomer_1.n_atoms} Atoms")

## Assigning names/types to the monomers

Keeping track of the different types of monomers in the system can be important for calculating pair-specific properties like RDFs.  When the molecules and monomers are generated, no types or names are assigned to the monomers.

Assigning names to individual monomers can be done using the `assign_types` function in the `Molecule` class. The important input parameter is defining a sequence. In the example below, lets assume that each molecule containing 20 monomers followed a `para-meta` sequence defined here as `"PM"`. 

In [ ]:
sequence = "PM"
for molecule in peek_system.molecules:
    molecule.sequence = sequence
    molecule.assign_types()

for i in range(5):
    print(peek_system.molecules[0].monomers[i].name)


In [ ]:
monomers = [i for i in peek_system.monomers()]
print(len(monomers))

### Properties at each structure level

In [ ]:
print(molecule_1.center)
print(monomer_1.center)
print(molecule_1.end_to_end_distance())

## Generating Components

There is a `Component` class that will allow you to group atoms together in structures that are smaller than the groupings in the `Monomer` class.

Right now, what is required is a dictionary of `name: indices`.

These indices are how the substructure exists within a single monomer, no the entire system.  For example, with PEEK which consists of 3 ring-like structures.  Two of them are identical, labeled `A` below, and the third is labeled `B`.  Since there are 2 `A` components, the indices passed into the dictionary are a list of 2 lists.

In [ ]:
comp_mapping = {
    "A": [[0, 1, 2, 3, 4, 20, 21], [5, 6, 7, 8, 9, 18, 19]],
    "B": [10, 11, 12, 13, 14, 15, 16, 17]
}

for monomer in peek_system.monomers():
    monomer.generate_components(index_mapping = comp_mapping)
    
print(len(molecule_1.components))
print(len(molecule_1.monomers[0].components))

#### Accessing each Component within the system.

In [ ]:
all_components = [comp for comp in peek_system.components()]
print(len(all_components))

### Parents are defined at the molecular level, and inherited down to each level of a substructure

In [ ]:
print(all_components[0].parent)
print(monomer_1.parent)
print(molecule_1)

## Generating Segments

So far, we've seen that structure groupings are automatically created at the molecule and monomer level. Then, with some inputs, we could create component-level structure groupings (smaller than a monomer). Using the `generate_segments()` method available in the `Molecule` class will let us create structure groupings at a scale of multiple monomers, but still smaller than a molecule.

Just like the `Components` class, this requires some input.  In this case, we have to define how many monomers are contained in each segment. For example, if we want segments to contain 4 monomers:

In [ ]:
for mol in peek_system.molecules:
    mol.generate_segments(monomers_per_segment = 4) # We're saying 4 monomers per segment

# Bond length and angle distributions

We can obtain information about the structure of the system at level more coarse than atomistic. For example, we might want to see the distribution of "bond" lengths and angles between monomers or segments.

The `System` class contains a function for each type of distribution.  The data is returned, and with the `plot` parameter you can see a plot of the distribution.

In [ ]:
bond_dist = peek_system.bond_length_distribution(use_monomers=True, plot=True)

In [ ]:
ang_dist = peek_system.bond_angle_distribution(use_monomers=True, plot=True)

## Bond lengths and angles between segments

The examples above used the bond distances and angles between monomers, but we can use the same methods on segments by changing the `user_monomers` and `use_segments` parameters in each function call.

In [ ]:
seg_bond_dist = peek_system.bond_length_distribution(use_monomers=False, use_segments=True, plot=True)

In [ ]:
seg_angle_dist = peek_system.bond_angle_distribution(use_monomers=False, use_segments=True, plot=True)

## Bond lengths and angles between components

In [ ]:
comp_bond_dist = peek_system.bond_length_distribution(use_components=True, use_monomers=False, plot=True)

In [ ]:
comp_ang_dist = peek_system.bond_angle_distribution(use_components=True, use_monomers=False, plot=True)

Both of the component bond length and angle distributions appear to be bimodal, meaning that each has 2 different characteristic values (distance and angle). We can try to look more closely at specific pairs for bond lengths and groups for bond angles by passing values to the `pair` and `group` parameters:

In [ ]:
comp_bond_dist = peek_system.bond_length_distribution(use_components=True,
                                                      pair=["A", "A"],
                                                      plot=True)

In [ ]:
comp_bond_dist = peek_system.bond_length_distribution(use_components=True,
                                                      pair=["A", "B"],
                                                      plot=True)

In [ ]:
comp_angle_dist = peek_system.bond_angle_distribution(use_components=True,
                                                      group=['B', 'A', 'A'],
                                                      plot=True)

# Saving a coarse-grained representation of the system to a snapshot

In [ ]:
snap = cmeutils.writers.write_snapshot(beads=[i for i in peek_system.monomers()])
cg_monomers_gsd = gsd.hoomd.open(name="cg_monomers_peek.gsd", mode="wb")
cg_monomers_gsd.append(snap)

In [ ]:
snap = cmeutils.writers.write_snapshot(beads=[i for i in peek_system.components()])
cg_components_gsd = gsd.hoomd.open(name="cg_rings_peek.gsd", mode="wb")
cg_components_gsd.append(snap)

# Coarse-graining with mBuild

We can use the center of mass methods and Molecule and Monomer classes to generate a coarse-grained representation of the UA system:

**NOTE:**
This requires that mbuild and py3Dmol are installed

`conda install -c conda-forge mbuild py3Dmol`

In [ ]:
import mbuild as mb

## Example: Using the Component() class to create a coarse-grained system

In [ ]:
import time

start = time.time()
cg_components = mb.Compound()
molecules = []
for mol in peek_system.molecules:
    mol_comp = mb.Compound()
    last_bead = None
    for mon_idx, mon in enumerate(mol.monomers):
        for comp_idx, comp in enumerate(mon.components):
            pos = comp.unwrapped_center / 10
            bead = mb.Compound(name=comp.name, pos=pos)
            mol_comp.add(bead)
            if mon_idx == 0 and comp_idx == 0:
                pass
            else:
                mol_comp.add_bond((bead, last_bead))
            last_bead = bead
    cg_components.add(mol_comp)
finish = time.time()
#cg_components.save("components_cg.gsd", overwrite=True)
print(finish - start)

cg_components.visualize(color_scheme = {"A": "blue", "B": "orange"}) 

## Example: Using the Monomer() class to create a coarse-grained system

Using mBuild, and the tools described above, we can generate a coarse-grained representation of the system where 1 bead is equivalent to 1 monomer from the united-atom system.

In [ ]:
start = time.time()
cg_monomers = mb.Compound()
for mol_idx, molecule in enumerate(peek_system.molecules):
    molecule_comp = mb.Compound(name=f"mol{mol_idx}")
    last_bead = None
    for mon_idx, monomer in enumerate(molecule.monomers):
        pos = monomer.unwrapped_center / 10
        bead = mb.Compound(name=monomer.name, pos=pos)
        molecule_comp.add(bead)
        if 0 < mon_idx < len(molecule.monomers):
            molecule_comp.add_bond((bead, last_bead))
        last_bead = bead
    cg_monomers.add(molecule_comp)
    
cg_monomers.save("monomers_cg.gsd", overwrite=True)
finish = time.time()
print(finish - start)

cg_monomers.visualize(color_scheme = {"P": "blue", "M": "orange"}) 

## Example: Using the Segment() class to create a coarse-grained system:

When creating a system object, the Molecule objects are automatically generated, and each time a `Molecule` class is instantiated, the `Monomer` objects are automatically created.

In order to create `Segment` instances, we will have to call the `generate_segments` method for each `Molecule` object.  The important parameter of input is defining how many monomers 1 segment consists of.

In [ ]:
start = time.time()
cg_segments = mb.Compound()

for mol_idx, molecule in enumerate(peek_system.molecules):
    molecule_comp = mb.Compound(name=f"mol{mol_idx}")
    last_bead = None
    for seg_idx, segment in enumerate(molecule.segments):
        bead_name = f"mol{mol_idx}_mon{seg_idx}"
        #pos = monomer.unwrapped_center * 3.3996695084235347
        pos = segment.unwrapped_center / 10
        bead = mb.Compound(name=bead_name, pos=pos)
        molecule_comp.add(bead)
        if 0 < seg_idx < len(molecule.monomers):
            molecule_comp.add_bond((bead, last_bead))
        last_bead = bead
    cg_segments.add(molecule_comp)

finish = time.time()
print(finish - start)

cg_segments.visualize()